# Topic modeling 1

In [1]:
import spacy
nlp = spacy.load('en')
import nltk
from nltk.corpus import stopwords
import pandas as pd
import numpy as np
import pickle

import re

import nltk
import re
import string
from nltk.corpus import stopwords

nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
lemmatizer = nltk.stem.WordNetLemmatizer()
w_tokenizer = nltk.tokenize.WhitespaceTokenizer()

import gensim
import pyLDAvis
from pyLDAvis import gensim as gensimvis
import spacy

import logging
from tqdm import tqdm
from pprint import pprint

[nltk_data] Downloading package wordnet to /home/ubuntu/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


**IMPORT DATA**

In [3]:
df = pd.read_pickle('pizza_sentences.pkl')


In [5]:
#df['cleaned'] = [w_tokenizer.tokenize(X.strip('][').replace(',','')) for X in df['cleaned']]
df.head()

,review_index,sentence_index,sentence,review_id,cleaned,sentiment,sentiment_val
0,5,0,I'll be the first to admit that I was not exci...,fdiNeiN_hoCxCMy2wTRW9g,"[admit, tavolta]","(1, Negative)",1.0
1,5,1,"Being a food snob, when a group of friends su...",fdiNeiN_hoCxCMy2wTRW9g,"[food, snob, group, friend, dinner, menu]","(1, Negative)",1.0
2,5,2,Im also not big on ordering pasta when I go out,fdiNeiN_hoCxCMy2wTRW9g,"[ordering, pasta]","(1, Negative)",1.0
3,5,3,"Alas, I was outnumbered",fdiNeiN_hoCxCMy2wTRW9g,[],"(2, Neutral)",2.0
4,5,4,Thank goodness! I ordered the sea bass special,fdiNeiN_hoCxCMy2wTRW9g,"[goodness, sea, bass]","(3, Positive)",3.0


In [6]:
df.shape

(728570, 7)

**SENTIMENT ANALYSIS**

In [6]:
from pycorenlp import StanfordCoreNLP
#in terminal:
#cd stanford-corenlp-full-2018-10-05
#java -mx5g -cp "*" edu.stanford.nlp.pipeline.StanfordCoreNLPServer -timeout 10000

nlp = StanfordCoreNLP('http://localhost:9000')

res = nlp.annotate(list(df.sentence)[1111],
                   properties={'annotators': 'sentiment',
                               'outputFormat': 'json',
                               'timeout': 1000,
                   })
for s in res["sentences"]:
    print("%d: '%s': %s %s" % (
        s["index"],
        " ".join([t["word"] for t in s["tokens"]]),
        s["sentimentValue"], s["sentiment"]))

0: '-LRB- I 'm not a lamb eater -RRB- I can say that I do n't even like Chicken Masala but I do like theirs': 1 Negative


In [7]:
def get_sentiment(sentence):
    try:
        res = nlp.annotate(sentence,
                       properties={'annotators': 'sentiment',
                                   'outputFormat': 'json',
                                   'timeout': 1000,
                       })
        result = (res['sentences'][0]['sentimentValue'], res['sentences'][0]['sentiment'])
    except Exception as e:
        result = ('error', e)
    return(result)

In [8]:
df = df[df['sentence'].isna()==False]
df['sentiment'] = [get_sentiment(X) for X in df['sentence']]
df['sentiment_val'] = [(X[0]) for X in df.sentiment]
df['sentiment_val'] = [int(X) if X != 'error' else np.nan for X in df.sentiment_val]


In [9]:
pd.to_pickle(df,'pizza_sentences.pkl')

**TOPIC MODELING**

In [10]:
new_stopwords = ['way', 'honey', 'otto', 'pick', 'end', 'vega', 'libretto', 'hour',
                 'place', 'vitos', 'wish', 'point', 'thats', 'bit', 'piece', 'word',
                 'review', 'door', 'guy', 'ill', 'alla', 'im', 'yelp', 'type', 'montreal',
                 'phone', 'aria', 'day', 'phoenix', 'star', 'ive', 'lot', 'need', 'table',
                 'thing', 'mario', 'year', 'spot', 'rest']
nlp = spacy.load('en', disable=['parser', 'ner'])
for s in new_stopwords:
    nlp.Defaults.stop_words.add(s)
    lex = nlp.vocab[s]
    lex.is_stop = True
    nlp.vocab[s].is_stop = True


In [17]:
def clean_stopwords(tokenized_text):
    tokenized_text = [t for t in tokenized_text if nlp.vocab[t].is_stop == False]
    return(tokenized_text)

In [19]:
def get_topics_lda(tokenized_text, num_topics):
    id2word = gensim.corpora.Dictionary(tokenized_text)
    corpus = [id2word.doc2bow(t) for t in tokenized_text]
    lda_model = gensim.models.ldamulticore.LdaMulticore(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=num_topics, 
                                           random_state=100,
                                           chunksize=100,
                                           passes=128,
                                           per_word_topics=True)
    return(lda_model)

In [20]:
df['cleaned'] = [clean_stopwords(t) for t in df['cleaned']]
id2word = gensim.corpora.Dictionary(df.cleaned)
corpus = [id2word.doc2bow(t) for t in df.cleaned]

In [ ]:
lda_model_whole = get_topics_lda(df.cleaned, 5)
lda_model_whole.print_topics(num_words=20)

In [ ]:
pd.to_pickle(lda_model_whole, 'lda_all.pkl')

In [2]:
lda_model_whole = pd.read_pickle('lda_all.pkl')

In [14]:
#get_document_topics = [lda_model_whole.get_document_topics(item) for item in corpus]

In [ ]:
get_document_topics

In [7]:
lda_model_whole.print_topics(num_words=20)

[(0,
  '0.081*"order" + 0.080*"pizza" + 0.039*"cheese" + 0.027*"pepperoni" + 0.026*"minute" + 0.019*"slice" + 0.018*"sausage" + 0.017*"meal" + 0.015*"option" + 0.015*"lunch" + 0.014*"mushroom" + 0.014*"drink" + 0.013*"area" + 0.013*"pepper" + 0.012*"beer" + 0.011*"atmosphere" + 0.011*"way" + 0.010*"wine" + 0.009*"thing" + 0.009*"piece"'),
 (1,
  '0.106*"food" + 0.086*"service" + 0.038*"delivery" + 0.032*"table" + 0.022*"review" + 0.022*"star" + 0.019*"guy" + 0.018*"hour" + 0.014*"phone" + 0.013*"bar" + 0.011*"year" + 0.009*"door" + 0.009*"bite" + 0.008*"room" + 0.008*"plate" + 0.008*"hotel" + 0.007*"party" + 0.006*"driver" + 0.006*"dining" + 0.006*"floor"'),
 (2,
  '0.052*"sauce" + 0.043*"restaurant" + 0.026*"location" + 0.025*"cheese" + 0.025*"menu" + 0.024*"customer" + 0.023*"wing" + 0.021*"crust" + 0.021*"pie" + 0.020*"topping" + 0.020*"experience" + 0.018*"taste" + 0.015*"tomato" + 0.014*"flavor" + 0.012*"spot" + 0.012*"quality" + 0.012*"lot" + 0.010*"manager" + 0.010*"mozzarella" 

**END NOTEBOOK**